In [1]:
from builtins import range
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from __future__ import print_function, division
from builtins import range, input

from future.utils import iteritems
from scipy.stats import norm
from scipy.stats import multivariate_normal as mvn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def getKaggleMNIST():
    train = pd.read_csv('/content/drive/MyDrive/PCA/digit-recognizer/train.csv').values.astype(np.float32)
    train = shuffle(train)

    Xtrain = train[:-1000,1:] / 255
    Ytrain = train[:-1000,0].astype(np.int32)

    Xtest  = train[-1000:,1:] / 255
    Ytest  = train[-1000:,0].astype(np.int32)
    return Xtrain, Ytrain, Xtest, Ytest

In [ ]:
class GaussianNB(object):
  def fit(self, X, Y, smoothing=1e-2):
    self.gaussians = dict()
    self.priors = dict()
    labels = set(Y)
    for c in labels:
      current_x = X[Y == c]
      self.gaussians[c] = {
        'mean': current_x.mean(axis=0),
        'var': current_x.var(axis=0) + smoothing,
      }
      self.priors[c] = float(len(Y[Y == c])) / len(Y)

  def score(self, X, Y):
    P = self.predict(X)
    return np.mean(P == Y)

  def predict(self, X):
    N, D = X.shape
    K = len(self.gaussians)
    P = np.zeros((N, K))
    for c, g in iteritems(self.gaussians):
      mean, var = g['mean'], g['var']
      P[:,c] = mvn.logpdf(X, mean=mean, cov=var) + np.log(self.priors[c])
    return np.argmax(P, axis=1)

In [ ]:
# get data
Xtrain, Ytrain, Xtest, Ytest = getKaggleMNIST()

# try NB by itself
model1 = GaussianNB()
model1.fit(Xtrain, Ytrain)

In [ ]:
print("NB train score:", model1.score(Xtrain, Ytrain))
print("NB test score:", model1.score(Xtest, Ytest))

NB train score: 0.8014634146341464
NB test score: 0.791


In [ ]:
#try NB with PCA first
pca = PCA(n_components=50)
Ztrain = pca.fit_transform(Xtrain)
Ztest = pca.transform(Xtest)

In [ ]:
model2 = GaussianNB()
model2.fit(Ztrain, Ytrain)
print("NB+PCA train score:", model2.score(Ztrain, Ytrain))
print("NB+PCA test score:", model2.score(Ztest, Ytest))

NB+PCA train score: 0.8730243902439024
NB+PCA test score: 0.861
